# Modeling an idealized tree crown surface
David Diaz

A flexible geometric form, a generalized ellipsoid can be used to approximate the three-dimensional crown surface of a tree. A two-dimensional ellipsoid for modeling crowns was originally proposed by Horn (1971):

$\frac{x^E}{a^E}+\frac{y^E}{b^E}=1$

A three-dimensional variant has been attributed to Pollock (1996), although a similar model was also described by Koop (1989). The figure below was produced by Sheng et al. (2001) and refers to this form as the Pollock model.

<img src='./images/Sheng et al 2001_Figure1.PNG' width='600' align='center'>

## Objective
In this notebook, we will implement an interactive 3D visualization of this geometric crown form, allowing the user to adjust the parameters and see how the crown shape evolves.

The interactions are enabled through the use of slider widgets, which show the user the values being selected for each parameter, and which then update the 3D visualization accordingly. These widgets and the 3D visualization are intended to provide a more intuitive sense of how the parameters affect the 3D model. 

Here is a hand-drawn mockup of the user interface we'll try to create.
<img src='./images/mockup_interactive_crown_modeler.png' width='300' align='center'>

In [1]:
import numpy as np
from ipyvolume import pylab as p3
from ipywidgets import (FloatSlider, IntSlider, 
                        VBox, interactive_output, 
                        Layout, Label)
from IPython.display import display

### Create a function to generate a crown surface and visualize it

In [2]:
def make_crown(crown_ratio, crown_radius, cc):
    top_ht = 100
    cr = crown_radius * top_ht
    ch = top_ht * crown_ratio
    bh = top_ht - ch
    xt, yt, zt = 0, 0, top_ht
    
    r = np.linspace(-cr,cr,25) # radii to generate surface valeus for
    theta = np.linspace(0,2*np.pi,32) # angles
    radius_matrix, theta_matrix = np.meshgrid(r,theta) # points within the crown
    x = radius_matrix * np.cos(theta_matrix) # x points within the crown
    y = radius_matrix * np.sin(theta_matrix) # y points within the crown
    
    # generate surface points of the crown using Sheng et al. (2001) equation
    z = ((1-(((x-xt)**2 + (y-yt)**2)**(cc/2))/(cr**cc))*(ch**cc) - ch + zt)**(1/cc) + bh
    
    p3.clear() # there seems to be a failure to clear the previous values from display
    
    # by default, ipyvolume shows Y-axis as up-down, and this can't apparently be modified
    # so switch out y and z in the plotting call
    p3.plot_wireframe(x,z,y, color='green') 
    p3.xlim(-top_ht,top_ht)
    p3.zlim(-top_ht,top_ht)
    p3.style.use('nobox')
    return p3.show()

### Create some user interface widgets

In [3]:
ratio = FloatSlider(min=0.0,max=1.0,step=0.01,value=0.75,
                    description='Crown Ratio:',
                    continuous_update=False,
                    layout=Layout(width='50%'))
curvature = FloatSlider(min=0.01,max=5,step=0.01, value=1.5,
                        description='Curvature:',
                        continuous_update=False,
                        layout=Layout(width='50%'))
radius = FloatSlider(min=0.01,max=1,step=.01,value=0.35,
                     description='Crown Radius:',
                     continuous_update=False,
                     layout=Layout(width='50%'),
                     style={'description_width': 'initial'})

# lay out the widgets
title = Label('Tree Crown Modeler')
ui = VBox([title,ratio,radius,curvature])

### Connect the widgets to our crown function

In [4]:
output = interactive_output(make_crown,
                            {'crown_ratio': ratio,
                             'crown_radius': radius,
                             'cc': curvature})

### And display it!

In [5]:
display(ui, output)

Output()

## Changes from storyboard through to implementation
It became apparent that allowing the user to change the tree's height did not seem to offer much of a difference in the visualization. The key parameters that seemed more important to visualize were the crown height (as ratio of total tree height), the width of the crown (as crown radius), and the curvature of the crown.

The visulization itself is still a bit jerkier than I would've liked. The `ipyvolume` library seems to be a work in progress. For example, I was unable to directly control the orientation of the axes in the 3D plot, so resorted to switching my Y and Z variables to have the crown be displayed top-up in the plot.

This interactive graphic took about 6-8 hours to complete, as this was the first time I had ever worked with `ipyvolume` or `ipywidgets`. Learning to use these packages and trouble-shooting the visualization took the most time. I spent a few hours just fiddling with different ways to try and refresh the display and scale the axes to try to make changes more apparent. 

This was also my first experience using [Binder](https://mybinder.org/) to share a Jupyter Notebook so that others could execute it without having to download any software. In the future, mocking up an interactive visualization like this would probably take more like 1-2 hours. 